In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import owlready2 as owl
from owlready2 import *
owlready2.reasoning.JAVA_MEMORY = 20000

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
torch.manual_seed(200)

In [ ]:
from create_pytorch_dataset import GeometricInterpretation, SCALE_FACTOR
from create_pytorch_dataset import entity_to_idx_vocab, concept_to_idx_vocab, role_to_idx_vocab, idx_to_entity_vocab, idx_to_concept_vocab, idx_to_role_vocab, idx_finder_dict
from create_pytorch_dataset import trainConceptDataset, testConceptDataset, trainRoleDataset, testRoleDataset, train_ConceptDataLoader, test_ConceptDataLoader, train_RoleDataLoader, test_RoleDataLoader

In [ ]:
from create_pytorch_dataset import X_concepts, X_roles

In [ ]:
print(f'There are {len(X_concepts)} in the concept dataset')
print(f'There are {len(X_roles)} in the role dataset')

In [ ]:
from FaithEL_model import FaithEL
from utils import save_model, train_model, plot_score_hak

In [ ]:
CENTROID_SCORE = True # When set to True, model scores assertion w.r.t distance to the centroid and to the moving parameter for concepts/roles
LR = 0.001
PHI = 1 # Weighs how far the concept parameter is allowed to move. The closer to 0, the less the parameter will move.
GAMMA = 1 # Weighs how far the individual parameter is allowed to move. The closer to 0, the less the parameter will move.
RADIUS = SCALE_FACTOR/2 + 0.1
EMB_DIM = 459

LOG_EPOCH = 100000
EVAL_TRAIN = True
EPOCHS = 250
EVAL_FREQ = 25

NEG_SAMPLING = True
PLOT_LOSS = True

DIM1 = 0
DIM2 = 1

loss_fn = nn.MSELoss()

In [ ]:
torch.manual_seed(269)

model = FaithEL(EMB_DIM, PHI, RADIUS, GAMMA,
                entity_to_idx_vocab, concept_to_idx_vocab, role_to_idx_vocab,
                )

optimizer = optim.Adam(model.parameters(), lr=LR)

In [ ]:
model_list = []

In [ ]:
train_loss_list, test_loss_list, train_hits_at_k_concept, test_hits_at_k_concept, train_hits_at_k_role, test_hits_at_k_role, = train_model(model, GeometricInterpretation,
                                                                                                                                           train_ConceptDataLoader, train_RoleDataLoader, test_ConceptDataLoader, test_RoleDataLoader,
                                                                                                                                           trainConceptDataset, testConceptDataset, trainRoleDataset, testRoleDataset,
                                                                                                                                           EPOCHS, LOG_EPOCH, EVAL_FREQ, EVAL_TRAIN, loss_fn, optimizer,
                                                                                                                                           idx_to_entity_vocab, entity_to_idx_vocab,
                                                                                                                                           idx_to_concept_vocab, concept_to_idx_vocab,
                                                                                                                                           idx_to_role_vocab, role_to_idx_vocab,
                                                                                                                                           CENTROID_SCORE, NEG_SAMPLING, PLOT_LOSS
                                                                                                                                           )

model_list.append(save_model(CENTROID_SCORE, LR, PHI, GAMMA, EMB_DIM, EPOCHS, LOG_EPOCH, EVAL_FREQ, EVAL_TRAIN,
                             loss_fn, model, optimizer, train_loss_list, test_loss_list, train_hits_at_k_concept, test_hits_at_k_concept, train_hits_at_k_role, test_hits_at_k_role))

In [ ]:
pos_list = -1
TOPK = 2

In [ ]:
plot_score_hak(model_list[pos_list]['test_hits_at_k_concept'], model_list[pos_list]['test_hits_at_k_role'], TOPK, model_list[pos_list]['epochs'], model_list[pos_list]['eval_freq'])
plot_score_hak(model_list[pos_list]['train_hits_at_k_concept'], model_list[pos_list]['train_hits_at_k_role'], TOPK, model_list[pos_list]['epochs'], model_list[pos_list]['eval_freq'])